In [1]:
import gc
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import ephem
import datetime
import numpy as np
import seaborn as sns
from scipy import signal
from IPython import display
from itertools import product
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.colors import LogNorm, DivergingNorm, SymLogNorm

sns.set()
sns.set_style("ticks")

In [3]:
from spectral_analysis.common_vars.time_slices import idx_t
from spectral_analysis.common_vars.directories import LUIGI_OUT_FOLDER,POSTPROCESS_OUT_FOLDER
from spectral_analysis.common_vars.regions import ids_regions,lats4id,lons4id,faces_regions
from spectral_analysis.isotropic_spectra.spectral_kinema import spectral_kinematics
from spectral_analysis.isotropic_spectra.co_spec import cospec_ab
from spectral_analysis.isotropic_spectra.isotropic import calc_ispec
from spectral_analysis.luigi_workflows.output import theta4idt,Theta4id,uv4idt,UV4id,tau4idt,Tau4id,VorticityGrid,H4id

In [4]:
%matplotlib widget

In [5]:
cmap_ranges = {
    "KE": [-1e-2,1e-2],
    "RV": [1e-12,1e-10],
    "DIV": [1e-12,1e-10],
    "RVDIV": [0.1,10]
}

In [6]:
xlims = [1/150,1/8]
xticks = [1/10,1/25,1/50,1/80,1/150]
xticksLabel = ['10','25','50','80','150']
yticks = [1/3,1/6,1/12,1/24,1/(24*7),1/(24*30)]
yticksLabel = ['3 h','6 h','12 h','1 d','1 w','1 mo']
# Lt plots
xticks_ = [1/10,1/20,1/30,1/40,1/50,1/75,1/100,1/150]
xticksLabel_ = ['10','20','30','40','50','75','100','150']

In [7]:
timevec
= idx_t["hours"]["JFM"]

In [8]:
def plot_cospectra(ki, omega, S, title, linthresh=1e-8, cmap='bwr', vminmax=None):
    plt.figure(figsize=(8,6))
    SS = (omega*S).T*ki
    if vminmax is None:
        vminmax = max(abs(np.min(SS)), np.max(SS))
    plt.pcolormesh(ki, omega, SS, norm = SymLogNorm(linthresh=linthresh,vmin=-1*vminmax,vmax=vminmax), cmap=cmap)
    #plt.clim(cmap_ranges["KE"])
    plt.xscale('log')
    plt.xticks(xticks,xticksLabel)
    plt.xlim(xlims)
    plt.yscale('log')
    plt.ylim([omega[1],omega[-100]])
    plt.yticks(yticks,yticksLabel)
    plt.colorbar()
    plt.xlabel('Horizontal scales [km]')
    plt.ylabel('Time scales')
    plt.title(title)
    plt.show()

In [9]:
def plotRegion(regionId):
    # Load vars
    grid = VorticityGrid(regionId)
    U,V = UV4id(regionId,timevec,0)
    qFlux = H4id(regionId,timevec)
    tauX,tauY = Tau4id(regionId,timevec,0)
    tau = np.sqrt(tauX**2 + tauY**2)

    # Rotational
    U_tfirst = np.moveaxis(U,-1,0)
    V_tfirst = np.moveaxis(V,-1,0)
    RV = grid.rv(U_tfirst,V_tfirst)
    RV = np.moveaxis(RV,0,-1)

    # Divergence
    DIV = grid.div(U_tfirst,V_tfirst)
    DIV = np.moveaxis(DIV,0,-1)

    # rot(tau)
    tauX_tfirst = np.moveaxis(tauX,-1,0)
    tauY_tfirst = np.moveaxis(tauY,-1,0)
    rotTau = grid.rv(tauX_tfirst,tauY_tfirst)
    rotTau = np.moveaxis(rotTau,0,-1)


    ## Cospectra oceQnet - RV/DIV
    co_qfluxRV,kx,ky,omega,dkx,dky,domega = cospec_ab(qFlux,RV,2,2,1)
    ki,co_qfluxRV_iso = calc_ispec(co_qfluxRV,kx,ky,omega)
    co_qfluxRV_iso.shape
    co_qfluxDIV,kx,ky,omega,dkx,dky,domega = cospec_ab(qFlux,DIV,2,2,1)
    ki,co_qfluxDIV_iso = calc_ispec(co_qfluxDIV,kx,ky,omega)
    co_qfluxDIV_iso.shape

    ## Cospectra tau - RV/DIV
    cospec_rvTau,kx,ky,omega,dkx,dky,domega = cospec_ab(RV,tau,2,2,1)
    ki,cospec_rv_iso = calc_ispec(cospec_rvTau,kx,ky,omega)
    cospec_rv_iso.shape
    cospec_divTau,kx,ky,omega,dkx,dky,domega = cospec_ab(DIV,tau,2,2,1)
    ki,cospec_div_iso = calc_ispec(cospec_divTau,kx,ky,omega)
    cospec_div_iso.shape

    ## Cospectra rot(tau) - RV/DIV
    cospec_rvRottau,kx,ky,omega,dkx,dky,domega = cospec_ab(RV,rotTau,2,2,1)
    ki,cospec_rvRottau_iso = calc_ispec(cospec_rvRottau,kx,ky,omega)
    cospec_rvRottau_iso.shape
    cospec_divRottau,kx,ky,omega,dkx,dky,domega = cospec_ab(DIV,rotTau,2,2,1)
    ki,cospec_divrotTau_iso = calc_ispec(cospec_divRottau,kx,ky,omega)
    cospec_divrotTau_iso.shape

    # Plot

    ## Cospectra oceQnet - RV/DIV
    plot_cospectra(ki, omega, co_qfluxRV_iso, "oceQnet - RV - {}".format(regionId), 1e-7)
    plot_cospectra(ki, omega, co_qfluxDIV_iso, "oceQnet - DIV - {}".format(regionId), 1e-7)

    ## Cospectra tau - RV/DIV
    plot_cospectra(ki, omega, cospec_rv_iso, "RV - |Tau| - {}".format(regionId), 1e-10)
    plot_cospectra(ki, omega, cospec_div_iso, "DIV - |Tau| - {}".format(regionId), 1e-10)

    ## Cospectra rot(tau) - RV/DIV
    plot_cospectra(ki, omega, cospec_rvRottau_iso, "RV - rot(Tau) - {}".format(regionId), 5e-14)
    plot_cospectra(ki, omega, cospec_divrotTau_iso, "DIV - rot(Tau) - {}".format(regionId), 5e-14)

In [10]:
plotRegion(730)

2020-11-19 08:09:19 INFO     UV shape (k=0): (290, 289, 2184)
2020-11-19 08:13:36 INFO     HF shape (k=0): (290, 289, 2184)
2020-11-19 08:15:38 INFO     TAUxy shape (k=0): (290, 289, 2184)


before fftn
290 289 2184
before fftn
290 289 2184
before fftn
290 289 2184
before fftn
290 289 2184
before fftn
290 289 2184
before fftn
290 289 2184


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
plotRegion(787)

2020-11-19 08:21:22 INFO     UV shape (k=0): (291, 289, 2184)
2020-11-19 08:25:42 INFO     HF shape (k=0): (291, 289, 2184)
2020-11-19 08:27:45 INFO     TAUxy shape (k=0): (291, 289, 2184)


before fftn
291 289 2184
before fftn
291 289 2184
before fftn
291 289 2184
before fftn
291 289 2184
before fftn
291 289 2184
before fftn
291 289 2184


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …